In [1]:
from osgeo import ogr
from  osgeo import ogr, osr
import os
import pickle

def filterCountyRings(key,union,filter_table=None):
    '''Returns a polygon with largest area in multiPolygon geometry object.'''
    if filter_table is None:
        return union
    if union.GetGeometryType() == ogr.wkbPolygon:
        return union
    #else
    filtered_geom=ogr.Geometry(ogr.wkbMultiPolygon)
    try:
        if union.GetGeometryType() == ogr.wkbMultiPolygon:
            max_index=0
            max_area=0.0
            for i in filter_table[key][1]: #select all in second list.
                geom = union.GetGeometryRef(i)
                #filtered_geom.AddGeometry(geom)
                if geom.GetArea() > max_area:
                    max_area = geom.GetArea()
                    max_index=i
            geom = union.GetGeometryRef(max_index)
            filtered_geom.AddGeometry(geom)
            
    except Exception,e:
        print("Exception in filterCountyRings()"),str(e)
    return filtered_geom

home =r'../'
srcshpfile = home + r'uscounties/cb_2016_us_county_500k.shp'
outputShapefile = home+ r'out/usa.shp'
filter_table_file = home+ r'out/filter_table.p'
state_dic ={}
state_cc ={}
exclude=['02','66','15','60','78','72','69'] #island states like Alaska,Guam,etc.

#open shape file.
shapef = ogr.Open(srcshpfile)
layer = shapef.GetLayer()
sourceSpatialRef = layer.GetSpatialRef()
#collect county rings for each of the states.
for fid in xrange(layer.GetFeatureCount()):
    fit= layer.GetFeature(fid)
    geom= fit.GetGeometryRef()
    #filter nasty boundaries.
    if fit.GetField('STATEFP') in exclude:
        continue
    try:
        if geom.GetGeometryType() == ogr.wkbPolygon:
            state_dic[fit.GetField('STATEFP')].AddGeometry(geom)
        elif geom.GetGeometryType() == ogr.wkbMultiPolygon:
            for i in range(geom.GetGeometryCount()):
                gp = geom.GetGeometryRef(i)
                state_dic[fit.GetField('STATEFP')].AddGeometry(gp)
    except:
        state_dic[fit.GetField('STATEFP')]=ogr.Geometry(ogr.wkbMultiPolygon)
        if geom.GetGeometryType() == ogr.wkbPolygon:
            state_dic[fit.GetField('STATEFP')].AddGeometry(geom)
        elif geom.GetGeometryType() == ogr.wkbMultiPolygon:
            for i in range(geom.GetGeometryCount()):
                gp = geom.GetGeometryRef(i)
                state_dic[fit.GetField('STATEFP')].AddGeometry(gp)


#load filter table, if not exist create one.filter table has state-id:[county-rings],[selective-rings]
try:
    filter_table= pickle.load( open(filter_table_file, "rb" ))
    print("filter_table file exists in: "),filter_table_file
except:
    filter_table={}
    for key, unionc in state_dic.items():
        try:
            union= unionc.UnionCascaded()
            filter_table[key]=[]
            if union.GetGeometryType() == ogr.wkbMultiPolygon:
                filter_table[key].append([j for j in range(union.GetGeometryCount())])#original-rings
                filter_table[key].append([j for j in range(union.GetGeometryCount())]) #put-items here to slect.
            else:
                filter_table[key].append([])
                filter_table[key].append([])
        except Exception, ex:
            print("Error while creating filter-table"),key, str(ex)
    
    pickle.dump(filter_table,open(filter_table_file, "wb" )) #end except
    print("filter_table dict. is saved in: "),filter_table_file
    
#saving layer
driver = ogr.GetDriverByName('ESRI Shapefile')
if os.path.exists(outputShapefile):
    driver.DeleteDataSource(outputShapefile)
outDataSet = driver.CreateDataSource(outputShapefile)
outLayer = outDataSet.CreateLayer("country",sourceSpatialRef,
                                  geom_type=ogr.wkbMultiPolygon)#name,SpatialRef,geom type
outLayer.CreateField(ogr.FieldDefn('country'),ogr.OFTInteger)
outLayer.CreateField(ogr.FieldDefn('area'),ogr.OFTInteger)
layerDefn = outLayer.GetLayerDefn()

'''merge counties to give states; and then merge states to give country.'''
print("key,#counties,#unionc,#union:")
countrygeom=ogr.Geometry(ogr.wkbMultiPolygon)
cnt=0
for key, unionc in state_dic.items():
    try:
        union= unionc.UnionCascaded()        
        print("Adding "),key,unionc.GetGeometryCount(), union.GetGeometryCount(),union.GetGeometryName()  
        #filter
        state = filterCountyRings(key,union,filter_table)
        if state.GetGeometryType() == ogr.wkbMultiPolygon:
            countrygeom.AddGeometry(state.GetGeometryRef(0))
        else: countrygeom.AddGeometry(state)
        state.Destroy()
    except Exception, e:
        print("except"),key,str(e)

print("total states"),cnt,countrygeom.GetGeometryCount()
country=countrygeom.UnionCascaded()
feat = ogr.Feature(layerDefn)

linearring= country.GetGeometryRef(0) #second linearing is a hole.
polygon=ogr.Geometry(ogr.wkbPolygon)
polygon.AddGeometry(linearring)
feat.SetGeometry(polygon) #second polygon is a hole.
    
feat.SetField('country',"0")
feat.SetField('area',country.GetArea())
outLayer.CreateFeature(feat)
feat.Destroy()
country.Destroy()
outDataSet.Destroy()
print("Country boundaries saved in "),outputShapefile

filter_table file exists in:  ../out/filter_table.p
key,#counties,#unionc,#union:
Adding  30 56 1 POLYGON
Adding  54 55 1 POLYGON
Adding  42 68 1 POLYGON
Adding  45 50 4 MULTIPOLYGON
Adding  36 83 18 MULTIPOLYGON
Adding  53 96 58 MULTIPOLYGON
Adding  32 17 1 POLYGON
Adding  24 41 18 MULTIPOLYGON
Adding  25 47 33 MULTIPOLYGON
Adding  26 193 111 MULTIPOLYGON
Adding  01 74 7 MULTIPOLYGON
Adding  20 105 1 POLYGON
Adding  21 121 2 MULTIPOLYGON
Adding  04 15 1 POLYGON
Adding  49 29 1 POLYGON
Adding  46 66 1 POLYGON
Adding  23 223 207 MULTIPOLYGON
Adding  08 78 1 POLYGON
Adding  09 15 8 MULTIPOLYGON
Adding  28 89 8 MULTIPOLYGON
Adding  29 115 1 POLYGON
Adding  40 77 1 POLYGON
Adding  41 43 8 MULTIPOLYGON
Adding  56 23 1 POLYGON
Adding  10 6 4 MULTIPOLYGON
Adding  39 99 12 MULTIPOLYGON
Adding  47 102 1 POLYGON
Adding  27 91 5 MULTIPOLYGON
Adding  06 72 13 MULTIPOLYGON
Adding  38 53 1 POLYGON
Adding  11 1 1 POLYGON
Adding  51 149 11 MULTIPOLYGON
Adding  13 166 2 MULTIPOLYGON
Adding  12 188 116 